In [1]:
import pandas as pd
chefmozparking = pd.read_csv("chefmozparking.csv")
geoplaces = pd.read_csv("geoplaces2.csv")
rating_final = pd.read_csv("rating_final.csv")

In [2]:
places = chefmozparking.merge(geoplaces, on = "placeID", how = "inner")

In [3]:
rating_final = rating_final.iloc[:,0:3]

In [4]:
restaurants = places.merge(rating_final, on="placeID", how="inner")
restaurants = restaurants[["price","parking_lot","smoking_area","other_services","dress_code","accessibility","rating"]]
restaurants.head()

,price,parking_lot,smoking_area,other_services,dress_code,accessibility,rating
0,medium,none,not permitted,Internet,informal,no_accessibility,0
1,medium,none,not permitted,Internet,informal,no_accessibility,2
2,medium,none,not permitted,Internet,informal,no_accessibility,1
3,medium,none,not permitted,Internet,informal,no_accessibility,1
4,low,none,none,none,informal,completely,2


In [5]:
restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1161 entries, 0 to 1160
Data columns (total 7 columns):
price             1161 non-null object
parking_lot       1161 non-null object
smoking_area      1161 non-null object
other_services    1161 non-null object
dress_code        1161 non-null object
accessibility     1161 non-null object
rating            1161 non-null int64
dtypes: int64(1), object(6)
memory usage: 72.6+ KB


In [6]:
restaurants.price.unique()

array(['medium', 'low', 'high'], dtype=object)

In [7]:
restaurants.parking_lot.unique()

array(['none', 'yes', 'public', 'valet parking'], dtype=object)

In [8]:
restaurants.parking_lot[restaurants.parking_lot == "valet parking"] = "valet_parking"

In [9]:
restaurants.smoking_area.unique()

array(['not permitted', 'none', 'only at bar', 'section', 'permitted'],
      dtype=object)

In [10]:
restaurants.smoking_area[restaurants.smoking_area == "not permitted"] = "not_permitted"
restaurants.smoking_area[restaurants.smoking_area == "only at bar"] = "only_at_bar"

/home/gunay/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
restaurants.other_services.unique()

array(['Internet', 'none', 'variety'], dtype=object)

In [12]:
restaurants.accessibility.unique()

array(['no_accessibility', 'completely', 'partially'], dtype=object)

In [13]:
restaurants.rating.unique()

array([0, 2, 1])

In [14]:
restaurants = pd.get_dummies(restaurants, drop_first=True)
restaurants.shape

(1161, 16)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(restaurants.drop(["rating"],axis = 1),
                                                    restaurants.rating,test_size = 0.001,random_state = 0)

In [17]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=3, random_state=0)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

# Model serialization 

In [19]:
import _pickle as pickle
import requests,json
pickle.dump(model,open("random_forest.pkl","wb"))

In [20]:
X = X_val.iloc[0,:].to_json(orient='split')
X = json.loads(X)

In [21]:
BASE_URL = "http://localhost:8080"
response = requests.post("{}/predict".format(BASE_URL), json = X)
print (response.json())

{'predicted_label': '1'}
